# English Premier League and the Ratings Percentage Index

This python notebook generates the English Premier League table ordered by the Ratings Percetage Index (RPI), using football match results data at [www.football-data.co.uk](http://www.football-data.co.uk/englandm.php). The RPI is a technique proposed by [The Tomkins Times](https://tomkinstimes.com/) subscriber Tim O'Brien to take account of the quality of opposition and is described [here](https://tomkinstimes.com/2016/11/comment-of-the-month-october-2016/). The solution is built into a simple web app.

The project uses [jupyter notebook](http://jupyter.org/index.html), [python](https://www.python.org/), [pandas](http://pandas.pydata.org/), [beautiful soup](https://www.crummy.com/software/BeautifulSoup/), [requests](http://docs.python-requests.org/en/master/), [streamlit](https://docs.streamlit.io/en/stable/) and [heroku](https://www.heroku.com/).

#### Notebook Change Log

In [21]:
%%html
<! left align the change log table in next cell >
<style>
table {float:left}
</style>

| Date          | Change Description |
| :------------ | :----------------- |
| 17th November 2016 | Initial baseline |
| 30th October 2017 | Updated *gen_prem_table_RPI()* to fix error in calculation for P (number of games played) |
| 5th November 2017 | Updated *gen_prem_table_RPI()* so that table is generated for 1st week |
| 31st August 2020 | Updated to convert to python3 |
| 31st August 2020 | Updated to move to streamlit rather than spyre |

## Set-up

Import the modules needed for the analysis.

In [22]:
import pandas as pd
import sys 
import requests
import datetime as dt
import pickle
import os
import bs4
from bs4 import BeautifulSoup
from IPython.display import Image

Print version numbers of key modules.

In [23]:
print(f"python version: {sys.version}")
print(f"pandas version: {pd.__version__}")
print(f"requests version: {requests.__version__}")
print(f"bs4 version: {bs4.__version__}")

python version: 3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]
pandas version: 1.1.1
requests version: 2.24.0
bs4 version: 4.9.1


## Generate the Premier League table with RPI

Start by defining some utility functions

In [24]:
def get_pl_master_data():
    """Return url of latest premier league results file and the date the file was last updated.
    
    Data source is www.football-data.co.uk.
    Format of returned date is "%Y-%m-%d" (the pandas default).
    """
    # scrape the data from football-data website
    URL_FD_ROOT = 'http://www.football-data.co.uk/'
    ENGLAND_LOCATION = 'englandm.php'
    PL_TEXT = 'Premier League'
    with requests.Session() as session:
        response = session.get(URL_FD_ROOT + ENGLAND_LOCATION)
        soup = BeautifulSoup(response.content, 'lxml')

        # scrape last updated date
        last_updated_tag = soup.find_all('i')[0]
        last_updated_date = last_updated_tag.text.split('Last updated: \t')[1]
        # set date format to be same as pandas default
        last_updated_date = dt.datetime.strptime(last_updated_date, '%d/%m/%y').strftime('%Y-%m-%d')
                                                                                         
        # scrape url of premier league results file
        latest_pl_results_file_tag = soup.findAll('a', href=True, text=PL_TEXT)[0]['href']
        url_latest_pl_results_file = URL_FD_ROOT + latest_pl_results_file_tag                                                                         
                                                                                     
    return(url_latest_pl_results_file, last_updated_date)

In [25]:
# check current latest
url_latest_pl_results_file, last_updated_date = get_pl_master_data()
print(f"PL results URL: {url_latest_pl_results_file}, last updated: {last_updated_date}")

PL results URL: http://www.football-data.co.uk/mmz4281/1920/E0.csv, last updated: 2020-07-26


In [26]:
    def get_pl_results_dataframe(update_cache=False):
        """Return latest premier league results as a dataframe and the date of the results data.

        Data source is www.football-data.co.uk.
        Cache data locally to avoid unnecessary calls to football-data website.
        Download results from master data source if local data is out of date.
        """

        LOCAL_RESULTS_DATA_FILE = 'data/E0.csv'
        PICKLE_FILE = 'save.p' # holds date of results data file

        # get master data source data
        url_latest_pl_results_file, master_results_data_date = get_pl_master_data()
        #print(f"master:  url_latest_pl_results_file is {url_latest_pl_results_file}, master_results_data_date is {master_results_data_date}")

        if update_cache:
            if os.path.exists(PICKLE_FILE):
                os.remove(PICKLE_FILE)

        # get local data source date
        if os.path.exists(PICKLE_FILE):
            local_results_data_date = pickle.load(open(PICKLE_FILE, 'rb'))
            #print(f"local: pickle file {PICKLE_FILE} exists, local_results_data_date is {local_results_data_date}")
        else:
            local_results_data_date = None
            #print(f"local: pickle file doesn't exists, local_results_data_date is {local_results_data_date}")

        if not local_results_data_date or local_results_data_date < master_results_data_date:
            #print('local results data out of date, updating from master results data file')
            parse_dates_col = ['Date']
            df_results = pd.read_csv(url_latest_pl_results_file, parse_dates=parse_dates_col, dayfirst=True)
            df_results.to_csv(LOCAL_RESULTS_DATA_FILE, index=False)
            local_results_data_date = master_results_data_date
            pickle.dump(local_results_data_date, open(PICKLE_FILE, 'wb'))
        else:
            print('local results data still latest')
            parse_dates_col = ['Date']
            df_results = pd.read_csv(LOCAL_RESULTS_DATA_FILE, parse_dates=parse_dates_col, dayfirst=True)

        return df_results, local_results_data_date

In [27]:
# check results
df_results, results_date = get_pl_results_dataframe()
print(f"results data date: {results_date}")
print(df_results.dtypes.head())
df_results.head()

results data date: 2020-07-26
Div                 object
Date        datetime64[ns]
Time                object
HomeTeam            object
AwayTeam            object
dtype: object


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,2019-08-09,20:00,Liverpool,Norwich,4,1,H,4,0,...,3.4,-2.2,1.9,2.0,1.9,2.0,2.0,2.1,1.9,2.0
1,E0,2019-08-10,12:30,West Ham,Man City,0,5,A,0,1,...,2.9,1.8,1.9,1.9,2.0,2.0,2.1,2.0,2.0,1.9
2,E0,2019-08-10,15:00,Bournemouth,Sheffield United,1,1,D,0,0,...,1.9,-0.5,1.9,1.9,2.0,1.9,2.0,2.0,2.0,1.9
3,E0,2019-08-10,15:00,Burnley,Southampton,3,0,H,0,0,...,1.7,0.0,1.9,2.0,1.9,2.0,1.9,2.1,1.9,2.0
4,E0,2019-08-10,15:00,Crystal Palace,Everton,0,0,D,0,0,...,1.7,0.2,1.8,2.1,2.0,2.0,2.0,2.1,2.0,1.9


In [28]:
# check forced update to cache
df_results, results_date = get_pl_results_dataframe(update_cache=True)
print(f"results data date: {results_date}")
print(df_results.dtypes.head())
df_results.head()

results data date: 2020-07-26
Div                 object
Date        datetime64[ns]
Time                object
HomeTeam            object
AwayTeam            object
dtype: object


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,2019-08-09,20:00,Liverpool,Norwich,4,1,H,4,0,...,3.4,-2.2,1.9,2.0,1.9,2.0,2.0,2.1,1.9,2.0
1,E0,2019-08-10,12:30,West Ham,Man City,0,5,A,0,1,...,2.9,1.8,1.9,1.9,2.0,2.0,2.1,2.0,2.0,1.9
2,E0,2019-08-10,15:00,Bournemouth,Sheffield United,1,1,D,0,0,...,1.9,-0.5,1.9,1.9,2.0,1.9,2.0,2.0,2.0,1.9
3,E0,2019-08-10,15:00,Burnley,Southampton,3,0,H,0,0,...,1.7,0.0,1.9,2.0,1.9,2.0,1.9,2.1,1.9,2.0
4,E0,2019-08-10,15:00,Crystal Palace,Everton,0,0,D,0,0,...,1.7,0.2,1.8,2.1,2.0,2.0,2.0,2.1,2.0,1.9


In [29]:
# check forced reload
local_data_source_date = '2016-11-05'
PICKLE_FILE = 'save.p'
pickle.dump(local_data_source_date, open(PICKLE_FILE, 'wb'))
print(pickle.load(open(PICKLE_FILE, 'rb')))
df_results, results_date = get_pl_results_dataframe()
print(f"results data date: {results_date}")
df_results.head()

2016-11-05
results data date: 2020-07-26


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,2019-08-09,20:00,Liverpool,Norwich,4,1,H,4,0,...,3.4,-2.2,1.9,2.0,1.9,2.0,2.0,2.1,1.9,2.0
1,E0,2019-08-10,12:30,West Ham,Man City,0,5,A,0,1,...,2.9,1.8,1.9,1.9,2.0,2.0,2.1,2.0,2.0,1.9
2,E0,2019-08-10,15:00,Bournemouth,Sheffield United,1,1,D,0,0,...,1.9,-0.5,1.9,1.9,2.0,1.9,2.0,2.0,2.0,1.9
3,E0,2019-08-10,15:00,Burnley,Southampton,3,0,H,0,0,...,1.7,0.0,1.9,2.0,1.9,2.0,1.9,2.1,1.9,2.0
4,E0,2019-08-10,15:00,Crystal Palace,Everton,0,0,D,0,0,...,1.7,0.2,1.8,2.1,2.0,2.0,2.0,2.1,2.0,1.9


In [30]:
df_results.tail()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
375,E0,2020-07-26,16:00,Leicester,Man United,0,2,A,0,0,...,2.0,0.5,1.9,2.0,1.9,2.0,1.9,2.0,1.9,2.0
376,E0,2020-07-26,16:00,Man City,Norwich,5,0,H,2,0,...,5.8,-3.8,2.0,1.9,2.0,1.9,2.1,1.9,2.0,1.8
377,E0,2020-07-26,16:00,Newcastle,Liverpool,1,3,A,1,1,...,2.4,1.0,1.9,2.0,1.9,2.0,2.0,2.0,1.9,1.9
378,E0,2020-07-26,16:00,Southampton,Sheffield United,3,1,H,0,1,...,2.0,-0.5,2.0,1.9,2.0,1.9,2.0,2.0,2.0,1.9
379,E0,2020-07-26,16:00,West Ham,Aston Villa,1,1,D,0,0,...,2.0,0.0,1.9,2.0,1.9,2.0,2.0,2.0,1.9,1.9


In [31]:
def validate_date(date_text):
    """Raise error if date format is not YYYY-MM-DD."""
    try:
        dt.datetime.strptime(date_text, '%Y-%m-%d')
    except ValueError:
        raise ValueError("Incorrect date format, should be YYYY-MM-DD")

In [32]:
assert validate_date('2016-11-14') == None
#assertRaises(ExpectedException, afunction, arg1, arg2)
try:
    validate_date('14-11-2016') == ValueError # produces error
    assert False, "unexpected pass! epected ValueError!"
except ValueError:
    assert True, "unexpected error! epected ValueError!"

In [33]:
def simple_date(date_text):
    """Return given date in format '%y-%m-%d' to '%d %b %y'."""
    validate_date(date_text)
    return (dt.datetime.strptime(date_text, '%Y-%m-%d').strftime('%d %b %y'))

In [34]:
print(simple_date('2016-6-2'))
print(simple_date('2017-1-31'))

02 Jun 16
31 Jan 17


Now produce function to generate the Prem League Table with RPI

Function Change Log
- October 2017, Updated gen_prem_table_RPI() to fix error in calculation for P (number of games played)
is listed but not yet played, so game data is not available
- November 2017, Updated gen_prem_table_RPI() so that table is generated for 1st week

In [35]:
def gen_prem_table_RPI(before_date=None, update_cache=False):
    """Return prem table with RPI at given before_date and return data source date."""
    
    results = []
    opponents_d = {}
    df_results, results_date = get_pl_results_dataframe(update_cache)
    
    # filter results in dataframe at given before_date
    if before_date:
        validate_date(before_date)
        df_results = df_results[df_results.Date <= before_date]
        
    for team in set(df_results.HomeTeam.tolist() + df_results.AwayTeam.tolist()):
        home_results = df_results[df_results['HomeTeam'] == team]
        home_played = len(home_results.index)
        home_win = home_results.FTR[home_results.FTR == 'H'].count()
        home_draw = home_results.FTR[home_results.FTR == 'D'].count()
        home_lose = home_results.FTR[home_results.FTR == 'A'].count()
        home_goals_for = home_results.FTHG.sum()
        home_goals_against = home_results.FTAG.sum()
        home_opponents = list(df_results[df_results.HomeTeam == team].AwayTeam.values)

        away_results = df_results[df_results['AwayTeam'] == team]
        away_played = len(away_results.index)
        away_win = away_results.FTR[away_results.FTR == 'A'].count()
        away_draw = away_results.FTR[away_results.FTR == 'D'].count()
        away_lose = away_results.FTR[away_results.FTR == 'H'].count()
        away_goals_for = away_results.FTAG.sum()
        away_goals_against = away_results.FTHG.sum()
        away_opponents = list(df_results[df_results.AwayTeam == team].HomeTeam.values)

        # add team opponents to dictionary
        team_opponents = home_opponents + away_opponents
        opponents_d[team] = team_opponents
        
        # create team results dictionary and add to results list
        result_d = {} 
        result_d['Team'] = team
        result_d['W'] = home_win + away_win
        result_d['D'] = home_draw + away_draw
        result_d['L'] = home_lose + away_lose
        result_d['GF'] = home_goals_for + away_goals_for
        result_d['GA'] = home_goals_against + away_goals_against
        result_d['GD'] = result_d['GF'] - result_d['GA']
        result_d['PTS'] = result_d['W']*3 + result_d['D']
        result_d['P'] = result_d['W'] + result_d['D'] + result_d['L']
        results.append(result_d) # append team result dictionary to list of results

    # create PL table dataframe from team results and sort by points (and then goal difference and goals for)
    # show date of data in Position column
    PLtable = pd.DataFrame(results, columns=['Team', 'P', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'PTS'])
    PLtable.sort_values(['PTS', 'GD', 'GF'], ascending=False, inplace=True)
    col_date = before_date if before_date else results_date
    pos_title = 'Position at {}'.format(simple_date(col_date))
    PLtable[pos_title] = range(1, len(PLtable)+1) # add new column for position, with highest points first
    PLtable.set_index([pos_title], inplace=True, drop=True) 
    #PLtable.reset_index(inplace=True)
    
    # Add RPI to the table
    PLtable['PTS%'] = 100*(PLtable.PTS/(PLtable.P*3))
    PLtable['OPP_PTS%'] = PLtable.apply(lambda row: PLtable[PLtable.Team.isin(opponents_d[row.Team])]['PTS%'].mean(), axis=1)
    PLtable['OPP_OPP_PTS%'] = PLtable.apply(lambda row: PLtable[PLtable.Team.isin(opponents_d[row.Team])]['OPP_PTS%'].mean(), axis=1)
    PLtable['RPI'] = (PLtable['PTS%']*.25 + PLtable['OPP_PTS%']*.50 + PLtable['OPP_OPP_PTS%']*.25)
    # replace nan with 0 just in case results are published when all games haven't yet been played 
    PLtable.fillna(0, inplace=True)  
    PLtable['RPI_Position'] = PLtable['RPI'].rank(ascending=False).astype(int)
    
    # return PL table with RPI, sorted by RPI and PTS percentage
    return(PLtable.sort_values(['RPI', 'PTS%'], ascending=False), results_date)

In [36]:
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI()
print(f"Premier league table ordered by RPI, date of results data: {results_date}")
PLtableRPI

local results data still latest
Premier league table ordered by RPI, date of results data: 2020-07-26


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 26 Jul 20,,,,,,,,,,,,,,
1,Liverpool,38,32,3,3,85,33,52,99,86.8,43.8,46.1,55.1,1
2,Man City,38,26,3,9,102,35,67,81,71.1,44.6,46.0,51.6,2
3,Man United,38,18,12,8,66,36,30,66,57.9,45.3,46.0,48.6,3
4,Chelsea,38,20,6,12,69,54,15,66,57.9,45.3,46.0,48.6,3
5,Leicester,38,18,8,12,67,41,26,62,54.4,45.5,46.0,47.9,5
6,Tottenham,38,16,11,11,61,47,14,59,51.8,45.7,46.0,47.3,6
7,Wolves,38,15,14,9,51,40,11,59,51.8,45.7,46.0,47.3,6
8,Arsenal,38,14,14,10,56,48,8,56,49.1,45.8,46.0,46.7,8
9,Sheffield United,38,14,12,12,39,39,0,54,47.4,45.9,46.0,46.3,9


In [38]:
# show index name
# note that this shows the last updated date (or requested date)
PLtableRPI.index.name

'Position at 26 Jul 20'

In [39]:
pd.set_option('precision', 1)
request_date = dt.datetime.today().strftime("%Y-%m-%d")
PLtableRPI, results_date = gen_prem_table_RPI(before_date=request_date, update_cache=False)
print(f"Premier league table ordered by RPI at {request_date}, date of results data: {results_date}")
PLtableRPI

local results data still latest
Premier league table ordered by RPI at 2020-09-04, date of results data: 2020-07-26


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 04 Sep 20,,,,,,,,,,,,,,
1,Liverpool,38,32,3,3,85,33,52,99,86.8,43.8,46.1,55.1,1
2,Man City,38,26,3,9,102,35,67,81,71.1,44.6,46.0,51.6,2
3,Man United,38,18,12,8,66,36,30,66,57.9,45.3,46.0,48.6,3
4,Chelsea,38,20,6,12,69,54,15,66,57.9,45.3,46.0,48.6,3
5,Leicester,38,18,8,12,67,41,26,62,54.4,45.5,46.0,47.9,5
6,Tottenham,38,16,11,11,61,47,14,59,51.8,45.7,46.0,47.3,6
7,Wolves,38,15,14,9,51,40,11,59,51.8,45.7,46.0,47.3,6
8,Arsenal,38,14,14,10,56,48,8,56,49.1,45.8,46.0,46.7,8
9,Sheffield United,38,14,12,12,39,39,0,54,47.4,45.9,46.0,46.3,9


In [40]:
pd.set_option('precision', 1)
request_date = '2019-10-24' # 9 games 2019-20
PLtableRPI, results_date = gen_prem_table_RPI(request_date)
print(f"Premier league table ordered by RPI at {request_date}, date of results data: {results_date}")
PLtableRPI

local results data still latest
Premier league table ordered by RPI at 2019-10-24, date of results data: 2020-07-26


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 24 Oct 19,,,,,,,,,,,,,,
1,Liverpool,9,8,1,0,21,7,14,25,92.6,43.6,48.5,57.1,1
3,Leicester,9,5,2,2,16,8,8,17,63.0,49.0,46.5,51.9,2
4,Chelsea,9,5,2,2,19,14,5,17,63.0,44.0,48.2,49.8,3
9,Sheffield United,9,3,3,3,8,7,1,12,44.4,50.2,45.2,47.5,4
2,Man City,9,6,1,2,29,9,20,19,70.4,37.4,44.6,47.5,5
5,Arsenal,9,4,3,2,13,12,1,15,55.6,43.6,46.2,47.3,6
14,Man United,9,2,4,3,10,9,1,10,37.0,52.3,45.0,46.7,7
8,Burnley,9,3,3,3,12,11,1,12,44.4,46.5,46.0,45.9,8
6,Crystal Palace,9,4,2,3,8,10,-2,14,51.9,42.8,45.4,45.7,9


In [41]:
pd.set_option('precision', 1)
request_date = '2019-10-31' # 10 games 2019-20
PLtableRPI, results_date = gen_prem_table_RPI(request_date)
print(f"Premier league table ordered by RPI at {request_date}, date of results data: {results_date}")
PLtableRPI

local results data still latest
Premier league table ordered by RPI at 2019-10-31, date of results data: 2020-07-26


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 31 Oct 19,,,,,,,,,,,,,,
1,Liverpool,10,9,1,0,23,8,15,28,93.3,43.3,48.2,57.1,1
3,Leicester,10,6,2,2,25,8,17,20,66.7,46.7,47.0,51.7,2
4,Chelsea,10,6,2,2,23,16,7,20,66.7,44.7,47.4,50.9,3
2,Man City,10,7,1,2,32,9,23,22,73.3,36.7,44.2,47.7,4
7,Man United,10,3,4,3,13,10,3,13,43.3,49.3,45.8,47.0,5
5,Arsenal,10,4,4,2,15,14,1,16,53.3,43.7,46.0,46.7,6
8,Sheffield United,10,3,4,3,9,8,1,13,43.3,49.3,44.2,46.5,7
11,Tottenham,10,3,3,4,16,15,1,12,40.0,48.7,44.2,45.4,8
13,Burnley,10,3,3,4,14,15,-1,12,40.0,48.0,45.4,45.4,9


## Building The Streamlit App

Useful reference material:
+ How to develop a Streamlit app, including tutorials - [streamlit](https://docs.streamlit.io/en/stable/).

Streamlit is a web app framework for providing a simple user interface and data viz for Python data projects. In simple terms the premrpi app involves:
1. using pandas (and related modules) to generate the data table.
2. creating a streamlit user interface to display the data table.

See [premrpi_app.py in the premrpi github repo](https://github.com/terrydolan/premrpi) for the app source code. 

Premrpi is deployed on the [heroku](https://www.heroku.com/) cloud platform.

Premrpi Web App History:
- The original 2016 solution used [Spyre](https://github.com/adamhajari/spyre), another simple web app framework from Adam Hajari
- Unfortunately Spyre is no longer supported in 2020 and hasn't been for some time
- Streamlit offers similar functionality to Spyre and is undergoing rapid development; it promises a simple framework for surfacing 'data apps' that may one day compete with RShiny

## Read the premrpi 'about' markdown

In [42]:
def read_premrpi_about_md(filename='premrpi_about.md'):
    """Read the premrpi about markdown file and return as a string."""
    with open(filename, "r") as f:
        premrpi_about_md = f.read()
    return premrpi_about_md

In [43]:
about_md = read_premrpi_about_md()
print(about_md)

premrpi v2.0.0

The premrpi app automatically generates the latest English Premier League table sorted by the Ratings Percentage Index (RPI).
The app updates itself when the Premier League results data changes at [www.football-data.co.uk](http://www.football-data.co.uk/englandm.php). 
 
The English Premier League table does not lie at the end of the season.
But earlier in the season the table does not tell the whole truth, even after 10+ games.
Some teams will have had more difficult starts, having played many of the stronger teams.
And other teams will have had easier starts, having played many of the weaker teams.
So how do you produce a league table that takes account of the quality of the opposition?
[The Tomkins Times](https://tomkinstimes.com/) subscriber Tim O'Brien (@Tjobrien17) proposed use of the RPI.
The idea is based on a technique used in American sports and is described by Tim [here](https://tomkinstimes.com/2016/11/comment-of-the-month-october-2016/).
        
The formul

## Edge Case Testing

Test at 2017 week 1, week 1 with partial data, week 2 and week 3

In [44]:
# gen_prem_table_RPI amended to parse an input test file
def gen_prem_table_RPI_test(before_date=None, update_cache=False, test_file='data/E0.csv'):
    """Return prem table with RPI at given before_date and return data source date."""
    
    results = []
    opponents_d = {}
    
    # generate results dataframe from test_file if provided
    if not test_file:
        df_results, results_date = get_pl_results_dataframe(update_cache)
    else: 
        # testing - read results from test file and set results_date to that in pickle file (save.p)
        print(f"*** Testing with {test_file}")
        df_results = pd.read_csv(test_file, parse_dates=['Date'], dayfirst=True)
        results_date = local_results_data_date = pickle.load(open('save.p', 'rb'))
    
    # filter results in dataframe at given before_date
    if before_date:
        validate_date(before_date)
        df_results = df_results[df_results.Date <= before_date]
    
    for team in set(df_results.HomeTeam.tolist() + df_results.AwayTeam.tolist()):
        home_results = df_results[df_results['HomeTeam'] == team]
        home_played = len(home_results.index)
        home_win = home_results.FTR[home_results.FTR == 'H'].count()
        home_draw = home_results.FTR[home_results.FTR == 'D'].count()
        home_lose = home_results.FTR[home_results.FTR == 'A'].count()
        home_goals_for = home_results.FTHG.sum()
        home_goals_against = home_results.FTAG.sum()
        home_opponents = list(df_results[df_results.HomeTeam == team].AwayTeam.values)

        away_results = df_results[df_results['AwayTeam'] == team]
        away_played = len(away_results.index)
        away_win = away_results.FTR[away_results.FTR == 'A'].count()
        away_draw = away_results.FTR[away_results.FTR == 'D'].count()
        away_lose = away_results.FTR[away_results.FTR == 'H'].count()
        away_goals_for = away_results.FTAG.sum()
        away_goals_against = away_results.FTHG.sum()
        away_opponents = list(df_results[df_results.AwayTeam == team].HomeTeam.values)

        # add team opponents to dictionary
        team_opponents = home_opponents + away_opponents
        opponents_d[team] = team_opponents
        
        # create team results dictionary and add to results list
        result_d = {} 
        result_d['Team'] = team
        result_d['W'] = home_win + away_win
        result_d['D'] = home_draw + away_draw
        result_d['L'] = home_lose + away_lose
        result_d['GF'] = home_goals_for + away_goals_for
        result_d['GA'] = home_goals_against + away_goals_against
        result_d['GD'] = result_d['GF'] - result_d['GA']
        result_d['PTS'] = result_d['W']*3 + result_d['D']
        result_d['P'] = result_d['W'] + result_d['D'] + result_d['L']
        results.append(result_d) # append team result dictionary to list of results

    # create PL table dataframe from team results and sort by points (and then goal difference and goals for)
    # show date of data in Position column
    PLtable = pd.DataFrame(results, columns=['Team', 'P', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'PTS'])
    PLtable.sort_values(['PTS', 'GD', 'GF'], ascending=False, inplace=True)
    col_date = before_date if before_date else results_date
    pos_title = 'Position at {}'.format(simple_date(col_date))
    PLtable[pos_title] = range(1, len(PLtable)+1) # add new column for position, with highest points first
    PLtable.set_index([pos_title], inplace=True, drop=True) 
    #PLtable.reset_index(inplace=True)
    
    # Add RPI to the table
    PLtable['PTS%'] = 100*(PLtable.PTS/(PLtable.P*3))
    PLtable['OPP_PTS%'] = PLtable.apply(lambda row: PLtable[PLtable.Team.isin(opponents_d[row.Team])]['PTS%'].mean(), axis=1)
    PLtable['OPP_OPP_PTS%'] = PLtable.apply(lambda row: PLtable[PLtable.Team.isin(opponents_d[row.Team])]['OPP_PTS%'].mean(), axis=1)
    PLtable['RPI'] = (PLtable['PTS%']*.25 + PLtable['OPP_PTS%']*.50 + PLtable['OPP_OPP_PTS%']*.25)
    PLtable.fillna(0, inplace=True)
    PLtable['RPI_Position'] = PLtable['RPI'].rank(ascending=False).astype(int)
    
    # return PL table with RPI, sorted by RPI and PTS percentage
    return(PLtable.sort_values(['RPI', 'PTS%'], ascending=False), results_date)

In [45]:
# Test using 2017
# expect all teams to have played 1 game
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2017-18_wk1_part.csv')
PLtableRPI

*** Testing with data/E0_2017-18_wk1_part.csv


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 26 Jul 20,,,,,,,,,,,,,,
1,Man United,1,1,0,0,4.0,0.0,4.0,3,100.0,0.0,100.0,50.0,7
2,Huddersfield,1,1,0,0,3.0,0.0,3.0,3,100.0,0.0,100.0,50.0,7
3,Man City,1,1,0,0,2.0,0.0,2.0,3,100.0,0.0,100.0,50.0,7
4,Arsenal,1,1,0,0,4.0,3.0,1.0,3,100.0,0.0,100.0,50.0,7
5,Burnley,1,1,0,0,3.0,2.0,1.0,3,100.0,0.0,100.0,50.0,7
6,West Brom,1,1,0,0,1.0,0.0,1.0,3,100.0,0.0,100.0,50.0,7
7,Everton,1,1,0,0,1.0,0.0,1.0,3,100.0,0.0,100.0,50.0,7
14,Leicester,1,0,0,1,3.0,4.0,-1.0,0,0.0,100.0,0.0,50.0,7
15,Chelsea,1,0,0,1,2.0,3.0,-1.0,0,0.0,100.0,0.0,50.0,7


In [46]:
# Test using 2017
# expect all teams to have played 1 game, with exception of 
# Newcastle vs Southampton which has not yet been played
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2017-18_wk1_part.csv')
PLtableRPI

*** Testing with data/E0_2017-18_wk1_part.csv


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 26 Jul 20,,,,,,,,,,,,,,
1,Man United,1,1,0,0,4.0,0.0,4.0,3,100.0,0.0,100.0,50.0,7
2,Huddersfield,1,1,0,0,3.0,0.0,3.0,3,100.0,0.0,100.0,50.0,7
3,Man City,1,1,0,0,2.0,0.0,2.0,3,100.0,0.0,100.0,50.0,7
4,Arsenal,1,1,0,0,4.0,3.0,1.0,3,100.0,0.0,100.0,50.0,7
5,Burnley,1,1,0,0,3.0,2.0,1.0,3,100.0,0.0,100.0,50.0,7
6,West Brom,1,1,0,0,1.0,0.0,1.0,3,100.0,0.0,100.0,50.0,7
7,Everton,1,1,0,0,1.0,0.0,1.0,3,100.0,0.0,100.0,50.0,7
14,Leicester,1,0,0,1,3.0,4.0,-1.0,0,0.0,100.0,0.0,50.0,7
15,Chelsea,1,0,0,1,2.0,3.0,-1.0,0,0.0,100.0,0.0,50.0,7


In [47]:
# Test using 2017
# expect all teams to have played 2 games
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2017-18_wk2.csv')
PLtableRPI

*** Testing with data/E0_2017-18_wk2.csv


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 26 Jul 20,,,,,,,,,,,,,,
13,Burnley,2,1,0,1,3,3,0,3,50.0,75.0,37.5,59.4,1
3,West Brom,2,2,0,0,2,0,2,6,100.0,25.0,79.2,57.3,2
8,Everton,2,1,1,0,2,1,1,4,66.7,58.3,45.8,57.3,3
14,Stoke,2,1,0,1,1,1,0,3,50.0,58.3,54.2,55.2,4
1,Man United,2,2,0,0,8,0,8,6,100.0,8.3,83.3,50.0,5
12,Chelsea,2,1,0,1,4,4,0,3,50.0,50.0,50.0,50.0,5
16,Bournemouth,2,0,0,2,0,3,-3,0,0.0,83.3,29.2,49.0,7
11,Arsenal,2,1,0,1,4,4,0,3,50.0,50.0,41.7,47.9,8
15,Swansea,2,0,1,1,0,4,-4,1,16.7,83.3,8.3,47.9,8


In [48]:
# Test using 2017
# expect all teams to have played 3 games
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2017-18_wk3.csv')
PLtableRPI

*** Testing with data/E0_2017-18_wk3.csv


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 26 Jul 20,,,,,,,,,,,,,,
6,Chelsea,3,2,0,1,6,4,2,6,66.7,44.4,58.0,53.4,1
10,Burnley,3,1,1,1,4,4,0,4,44.4,63.0,40.7,52.8,2
12,Everton,3,1,1,1,2,3,-1,4,44.4,63.0,38.3,52.2,3
1,Man United,3,3,0,0,10,0,10,9,100.0,25.9,54.3,51.5,4
5,West Brom,3,2,1,0,3,1,2,7,77.8,29.6,61.7,49.7,5
11,Stoke,3,1,1,1,2,2,0,4,44.4,51.9,48.1,49.1,6
13,Swansea,3,1,1,1,2,4,-2,4,44.4,51.9,44.4,48.1,7
9,Tottenham,3,1,1,1,4,3,1,4,44.4,48.1,49.4,47.5,8
2,Liverpool,3,2,1,0,8,3,5,7,77.8,29.6,49.4,46.6,9


In [49]:
# Test using 2017
# expect all teams to have played 10 games
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2017-18_wk10.csv')
PLtableRPI

*** Testing with data/E0_2017-18_wk10.csv


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 26 Jul 20,,,,,,,,,,,,,,
1,Man City,10,9,1,0,35,6,29,28,93.3,39.3,49.9,55.5,1
6,Liverpool,10,4,4,2,17,16,1,16,53.3,54.3,43.7,51.4,2
4,Chelsea,10,6,1,3,18,10,8,19,63.3,46.7,47.3,51.0,3
3,Tottenham,10,6,2,2,19,7,12,20,66.7,44.0,46.9,50.4,4
2,Man United,10,7,2,1,23,4,19,23,76.7,37.7,47.9,50.0,5
7,Burnley,10,4,4,2,9,9,0,16,53.3,46.7,46.9,48.4,6
8,Watford,10,4,3,3,15,18,-3,15,50.0,47.7,46.2,47.9,7
5,Arsenal,10,6,1,3,19,13,6,19,63.3,39.3,49.2,47.8,8
14,Stoke,10,3,2,5,11,20,-9,11,36.7,52.0,44.5,46.3,9


In [50]:
# Test using 2017
# expect all teams to have played 10 games with exception of Burnley and Newcastle who have played 9
# Burnley vs Newcastle not included as played on Monday 30th October
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2017-18_wk10_part.csv')
PLtableRPI

*** Testing with data/E0_2017-18_wk10_part.csv


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 26 Jul 20,,,,,,,,,,,,,,
1,Man City,10,9,1,0,35.0,6.0,29.0,28,93.3,38.8,49.9,55.2,1
6,Liverpool,10,4,4,2,17.0,16.0,1.0,16,53.3,54.3,43.7,51.4,2
4,Chelsea,10,6,1,3,18.0,10.0,8.0,19,63.3,46.1,47.3,50.7,3
3,Tottenham,10,6,2,2,19.0,7.0,12.0,20,66.7,44.0,46.8,50.4,4
2,Man United,10,7,2,1,23.0,4.0,19.0,23,76.7,37.7,48.0,50.0,5
7,Watford,10,4,3,3,15.0,18.0,-3.0,15,50.0,47.7,46.2,47.9,6
5,Arsenal,10,6,1,3,19.0,13.0,6.0,19,63.3,39.3,49.2,47.8,7
10,Burnley,9,3,4,2,8.0,9.0,-1.0,13,48.1,47.2,46.7,47.3,8
14,Stoke,10,3,2,5,11.0,20.0,-9.0,11,36.7,52.5,44.3,46.5,9


In [51]:
#E0_2016-17_final.csv

In [52]:
# Test using 2016
# expect all teams to have played 38 games
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2016-17_final.csv')
PLtableRPI

*** Testing with data/E0_2016-17_final.csv


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 26 Jul 20,,,,,,,,,,,,,,
1,Chelsea,38,30,3,5,85,33,52,93,81.6,44.5,46.4,54.2,1
2,Tottenham,38,26,8,4,86,26,60,86,75.4,44.8,46.4,52.9,2
3,Man City,38,23,9,6,80,39,41,78,68.4,45.2,46.4,51.3,3
4,Liverpool,38,22,10,6,78,42,36,76,66.7,45.2,46.4,50.9,4
5,Arsenal,38,23,6,9,77,44,33,75,65.8,45.3,46.4,50.7,5
6,Man United,38,18,15,5,54,29,25,69,60.5,45.6,46.4,49.5,6
7,Everton,38,17,10,11,62,44,18,61,53.5,45.9,46.3,47.9,7
8,Southampton,38,12,10,16,41,48,-7,46,40.4,46.6,46.3,45.0,8
9,Bournemouth,38,12,10,16,55,67,-12,46,40.4,46.6,46.3,45.0,8


In [53]:
# Test using 2016
# expect all teams to have played 9 games - see expected results from TTT below
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(before_date='2016-10-25', test_file='data/E0_2016-17_final.csv')
PLtableRPI

*** Testing with data/E0_2016-17_final.csv


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 25 Oct 16,,,,,,,,,,,,,,
3,Liverpool,9,6,2,1,20,11,9,20,74.1,47.3,47.6,54.1,1
4,Chelsea,9,6,1,2,19,9,10,19,70.4,44.9,48.1,52.0,2
2,Arsenal,9,6,2,1,19,9,10,20,74.1,42.8,47.5,51.8,3
1,Man City,9,6,2,1,20,9,11,20,74.1,40.7,46.1,50.4,4
5,Tottenham,9,5,4,0,13,4,9,19,70.4,43.6,43.9,50.4,5
7,Man United,9,4,2,3,13,12,1,14,51.9,50.6,45.7,49.7,6
10,Bournemouth,9,3,3,3,12,12,0,12,44.4,48.6,44.1,46.4,7
12,Leicester,9,3,2,4,11,15,-4,11,40.7,49.0,46.8,46.4,8
14,Burnley,9,3,1,5,8,13,-5,10,37.0,50.2,46.3,45.9,9


In [54]:
# Test using 2016
# expect all teams to have played 10 games - see expected results from TTT below
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(before_date='2016-11-1', test_file='data/E0_2016-17_final.csv')
PLtableRPI

*** Testing with data/E0_2016-17_final.csv


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 01 Nov 16,,,,,,,,,,,,,,
3,Liverpool,10,7,2,1,24,13,11,23,76.7,45.3,47.4,53.7,1
4,Chelsea,10,7,1,2,21,9,12,22,73.3,44.7,47.4,52.5,2
2,Arsenal,10,7,2,1,23,10,13,23,76.7,40.0,47.8,51.1,3
1,Man City,10,7,2,1,24,9,15,23,76.7,39.0,46.3,50.2,4
5,Tottenham,10,5,5,0,14,5,9,20,66.7,44.3,44.6,50.0,5
8,Man United,10,4,3,3,13,12,1,15,50.0,50.0,45.5,48.9,6
11,Leicester,10,3,3,4,12,16,-4,12,40.0,50.0,46.8,46.7,7
6,Everton,10,5,3,2,15,8,7,18,60.0,40.3,45.1,46.4,8
14,Burnley,10,3,2,5,8,13,-5,11,36.7,51.0,46.3,46.2,9


In [55]:
#Note: results after 10 games in 2016...
Image(url= "https://tomkinstimes.com/wp-content/uploads/2016/11/RPI-by-Tim-OBrien.png")